# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/weather_data.csv")

weather_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 10 columns):
Unnamed: 0    531 non-null int64
City          531 non-null object
Cloudiness    531 non-null int64
Country       528 non-null object
Date          531 non-null int64
Humidity      531 non-null int64
Lat           531 non-null float64
Lng           531 non-null float64
Max Temp      531 non-null float64
Wind Speed    531 non-null float64
dtypes: float64(4), int64(4), object(2)
memory usage: 41.6+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]]

# Convert Humidity to float and store
humidity = weather_data['Humidity'].astype(float)

In [4]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

narrowed_city_df = weather_data.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,47,47,Castro,0,CL,1577895837,53,-42.48,-73.76,71.60,6.93
1,91,91,Coahuayana,0,MX,1577896123,48,18.62,-100.35,75.81,0.34
2,228,228,Najran,0,SA,1577896278,36,17.54,44.22,75.20,4.70
3,322,322,Ancud,0,CL,1577896385,53,-41.87,-73.83,71.60,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

# create hotel_df with hotel name column
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,47,Castro,0,CL,1577895837,53,-42.48,-73.76,71.60,6.93,NaN
1,91,Coahuayana,0,MX,1577896123,48,18.62,-100.35,75.81,0.34,NaN
2,228,Najran,0,SA,1577896278,36,17.54,44.22,75.20,4.70,NaN
3,322,Ancud,0,CL,1577896385,53,-41.87,-73.83,71.60,6.93,NaN


In [7]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#checker variable
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,47,Castro,0,CL,1577895837,53,-42.48,-73.76,71.60,6.93,Hotel Esmeralda
1,91,Coahuayana,0,MX,1577896123,48,18.62,-100.35,75.81,0.34,NaN
2,228,Najran,0,SA,1577896278,36,17.54,44.22,75.20,4.70,فندق حياة نجران
3,322,Ancud,0,CL,1577896385,53,-41.87,-73.83,71.60,6.93,HOTEL ANCUD DON LUCAS


In [10]:
#clean up DataSet by remooving NAN
hotel_df_clean = hotel_df.dropna()
hotel_df_clean


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,47,Castro,0,CL,1577895837,53,-42.48,-73.76,71.6,6.93,Hotel Esmeralda
2,228,Najran,0,SA,1577896278,36,17.54,44.22,75.2,4.70,فندق حياة نجران
3,322,Ancud,0,CL,1577896385,53,-41.87,-73.83,71.6,6.93,HOTEL ANCUD DON LUCAS


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))